# Examples

Here follows examples of how to use the utility Python functions contained in `utils.py`.

In [2]:
import json
from utils import *

import warnings

# Suppress FutureWarning on pandas
warnings.filterwarnings("ignore", category=FutureWarning)

# Load ferries data
with open("ferries.json", "r") as file:
    ferries_data = json.load(file)

## `fetch_vessel_data`

Fetches historical vessel data from PONTOS-HUB through the REST API.
The function requires a specified time range because it is NOT possible to access all the data for a specific vessel with a single request. It is just too much data and the REST API has a limit of returning max 1 million rows per request.

The example below fetches the data of the ferry "Fragancia" as it is stored in the hub for a time range of 10 minutes. 

    NOTE: `fetch_vessel_data` assumes the time is in Coordinated Universal Time (UTC) and not Central European Time (CET) or similar. The timestamps in the returned vessel data are also in UTC.

In [5]:
# PONTOS vessel id for ferry "Fragancia"
vessel_id = "mmsi_265558290"

# Time range for fetching data (10 minutes)
start_time = "2024-11-01 12:00:00"
end_time = "2024-11-01 12:10:00"

# Fetch vessel data
vessel_data = fetch_vessel_data(vessel_id, start_time, end_time)

# Print the first 5 data points
print(f"Vessel data contains {len(vessel_data)} data points within the 10 minutes interval.")
unique_parameters = set(data_point['parameter_id'] for data_point in vessel_data)


# Formatting the unique measurement output
unique_parameters_str = "\n\t".join(unique_parameters)
print(f"The unique measurements available are:\n\t{unique_parameters_str}")

print("The first data point is:")
print(vessel_data[0])

Vessel data contains 7200 data points within the 10 minutes interval.
The unique measurements available are:
	enginemain_fuelcons_lph_3
	enginemain_fuelcons_lph_2
	enginemain_speed_rpm_2
	positioningsystem_longitude_deg_1
	enginemain_speed_rpm_4
	positioningsystem_sog_kn_1
	enginemain_fuelcons_lph_1
	positioningsystem_latitude_deg_1
	positioningsystem_cog_deg_1
	enginemain_speed_rpm_1
	enginemain_fuelcons_lph_4
	enginemain_speed_rpm_3
The first data point is:
{'time': '2024-11-01T12:00:00+00:00', 'parameter_id': 'enginemain_fuelcons_lph_1', 'value': 4.7999997}


To facilate the handling of the data, the `fetch_vessel_data` function can use the PONTOS-HUB resources that return data averaged within different "time buckets". This functionality is controlled by the `time_bucket` argument. The examples below demonstrate.

In [6]:
# Fetch vessel data, averaged within a 5 seconds time bucket
vessel_data = fetch_vessel_data(vessel_id, start_time, end_time, time_bucket="5 seconds")
print(f"Vessel data contains {len(vessel_data)} data points within the 10 minutes interval.")

# Fetch vessel data, averaged within a 30 seconds time bucket
vessel_data = fetch_vessel_data(vessel_id, start_time, end_time, time_bucket="30 seconds")
print(f"Vessel data contains {len(vessel_data)} data points within the 10 minutes interval.")

# Fetch vessel data, averaged within a 1 minute time bucket
vessel_data = fetch_vessel_data(vessel_id, start_time, end_time, time_bucket="1 minute")
print(f"Vessel data contains {len(vessel_data)} data points within the 10 minutes interval.")

# Fetch vessel data, averaged within a 5 minutes time bucket
vessel_data = fetch_vessel_data(vessel_id, start_time, end_time, time_bucket="5 minutes")
print(f"Vessel data contains {len(vessel_data)} data points within the 10 minutes interval.")

# Fetch vessel data, averaged within a 10 minutes time bucket
vessel_data = fetch_vessel_data(vessel_id, start_time, end_time, time_bucket="10 minutes")
print(f"Vessel data contains {len(vessel_data)} data points within the 10 minutes interval.")


Vessel data contains 1440 data points within the 10 minutes interval.
Vessel data contains 240 data points within the 10 minutes interval.
Vessel data contains 120 data points within the 10 minutes interval.
Vessel data contains 24 data points within the 10 minutes interval.
Vessel data contains 12 data points within the 10 minutes interval.


As expected, the number of data points decreases as the size of the time bucket increases. 

The `fetch_vessel_data` function can also request specific measurements. The `parameter_ids` argument takes a list of strings that are to be matched with the the tags of the available measurements. The examples below demonstrates this.

In [8]:
# Fetch vessel data, with only measurement tags that include the words 'latitude' and 'longitude', averaged within a 10 minutes time bucket
vessel_data = fetch_vessel_data(
    vessel_id, 
    start_time, 
    end_time, 
    time_bucket="10 minutes",
    parameter_ids=["latitude", "longitude"]
)
print("With parameter_ids=['latitude', 'longitude'] ...")
print(f"Vessel data contains {len(vessel_data)} data points within the 10 minutes interval.")
measurements_str = "\n\t".join(data_point['parameter_id'] for data_point in vessel_data)
print(f"The measurements available are:\n\t{measurements_str}\n")

# Fetch vessel data, only the measurements with the tag 'enginemain', averaged within a 10 minutes time bucket
vessel_data = fetch_vessel_data(
    vessel_id, 
    start_time, 
    end_time, 
    time_bucket="10 minutes",
    parameter_ids=["enginemain"]
)
print("With parameter_ids=['enginemain'] ...")
measurements_str = "\n\t".join(data_point['parameter_id'] for data_point in vessel_data)
print(f"Vessel data contains {len(vessel_data)} data points within the 10 minutes interval.")
print(f"The measurements available are:\n\t{measurements_str}\n")

# Fetch vessel data, only the measurements with the tag 'enginemain_fuelcons_lph_1', averaged within a 10 minutes time bucket
vessel_data = fetch_vessel_data(
    vessel_id, 
    start_time, 
    end_time, 
    time_bucket="10 minutes",
    parameter_ids=["enginemain_fuelcons_lph_1"]
)
print("With parameter_ids=['enginemain_fuelcons_lph_1'] ...")
measurements_str = "\n\t".join(data_point['parameter_id'] for data_point in vessel_data)
print(f"Vessel data contains {len(vessel_data)} data points within the 10 minutes interval.")
print(f"The measurements available are:\n\t{measurements_str}\n")

With parameter_ids=['latitude', 'longitude'] ...
Vessel data contains 2 data points within the 10 minutes interval.
The measurements available are:
	positioningsystem_latitude_deg_1
	positioningsystem_longitude_deg_1

With parameter_ids=['enginemain'] ...
Vessel data contains 8 data points within the 10 minutes interval.
The measurements available are:
	enginemain_fuelcons_lph_1
	enginemain_fuelcons_lph_2
	enginemain_fuelcons_lph_3
	enginemain_fuelcons_lph_4
	enginemain_speed_rpm_1
	enginemain_speed_rpm_2
	enginemain_speed_rpm_3
	enginemain_speed_rpm_4

With parameter_ids=['enginemain_fuelcons_lph_1'] ...
Vessel data contains 1 data points within the 10 minutes interval.
The measurements available are:
	enginemain_fuelcons_lph_1



## `transform_vessel_data_to_dataframe`

Transforms vessel data into a Pandas DataFrame where each row corresponds to a timestamp. If not averaged, the resolution of the timestamps is 1 second and the first measurment in that second is used in the dataframe. If averaged, the resolution of the timestamp is the size of the time bucket. Also, if averaged, the `avg_` prefixes are removed.

The examples below demonstrate.

In [9]:
# Fetch vessel data without averaging
vessel_data = fetch_vessel_data(
    vessel_id, 
    start_time, 
    end_time, 
    parameter_ids=["latitude","longitude","sog","fuelcons"]
    )
df = transform_vessel_data_to_dataframe(vessel_data)
print(f"The data contains {len(df)} rows.")
df.head(5)

The data contains 600 rows.


parameter_id,time,enginemain_fuelcons_lph_1,enginemain_fuelcons_lph_2,enginemain_fuelcons_lph_3,enginemain_fuelcons_lph_4,positioningsystem_latitude_deg_1,positioningsystem_longitude_deg_1,positioningsystem_sog_kn_1
id,,,,,,,,
0,2024-11-01 12:00:00+00:00,4.8,4.7,4.7,4.8,59.395300,18.441317,0.0
1,2024-11-01 12:00:01+00:00,4.7,4.5,4.7,4.8,59.395300,18.441317,0.0
2,2024-11-01 12:00:02+00:00,4.6,4.4,4.7,4.8,59.395300,18.441317,0.0
3,2024-11-01 12:00:03+00:00,4.6,4.4,4.7,4.8,59.395300,18.441317,0.0
4,2024-11-01 12:00:04+00:00,4.5,4.3,4.7,4.8,59.395306,18.441317,0.0


In [10]:
# Fetch vessel data
vessel_data = fetch_vessel_data(
    vessel_id, 
    start_time,
    end_time,
    time_bucket="30 seconds",
    parameter_ids=["latitude","longitude","sog","fuelcons"]
)
df = transform_vessel_data_to_dataframe(vessel_data)

df.head(5)

parameter_id,time,enginemain_fuelcons_lph_1,enginemain_fuelcons_lph_2,enginemain_fuelcons_lph_3,enginemain_fuelcons_lph_4,positioningsystem_latitude_deg_1,positioningsystem_longitude_deg_1,positioningsystem_sog_kn_1
id,,,,,,,,
0,2024-11-01 12:00:14.500000+00:00,4.380000,4.180000,4.703334,4.803333,59.395307,18.441321,0.0
1,2024-11-01 12:00:44.500000+00:00,4.363333,4.153333,4.620000,4.780000,59.395315,18.441326,0.0
2,2024-11-01 12:01:14.500000+00:00,4.400000,4.200000,4.543334,4.763333,59.395319,18.441328,0.0
3,2024-11-01 12:01:44.500000+00:00,4.400000,4.200000,4.596667,4.786666,59.395315,18.441328,0.0
4,2024-11-01 12:02:14.500000+00:00,4.400000,4.200000,4.600000,4.786666,59.395301,18.441330,0.0


## `plot_paths`

Plots a series of paths on a map using the pydeck library where a 'path' is a list of latitude, longitude tuples/lists. The example below demonstrates.

In [11]:
# Fetch vessel data, averaged within a 30 seconds time bucket
vessel_data = fetch_vessel_data(
    vessel_id, 
    "2024-11-01 12:00:00",
    "2024-11-01 12:30:00",
    time_bucket="30 seconds",
    parameter_ids=["positioningsystem_latitude_deg_1","positioningsystem_longitude_deg_1"]
)
df = transform_vessel_data_to_dataframe(vessel_data)

# Create a path
path = df[["positioningsystem_latitude_deg_1", "positioningsystem_longitude_deg_1"]].values.tolist()

# Plot the path (path is wrapped in a list because plot_paths expects a list of paths)
plot_paths([path])

{
  "initialViewState": {
    "latitude": 59.39556081833334,
    "longitude": 18.444181893888885,
    "zoom": 15
  },
  "layers": [
    {
      "@@type": "PathLayer",
      "data": [
        {
          "color": [
            31,
            119,
            180
          ],
          "path": [
            [
              18.4413208,
              59.3953072
            ],
            [
              18.441326,
              59.395315
            ],
            [
              18.4413276,
              59.3953185
            ],
            [
              18.4413278,
              59.395314666666664
            ],
            [
              18.4413304,
              59.3953014
            ],
            [
              18.4413284,
              59.3953
            ],
            [
              18.441333933333333,
              59.39529686666667
            ],
            [
              18.4413413,
              59.3952924
            ],
            [
              18.441339033333332,
              59.39529
            ],
            [
              18.4413436,
              59.3952885
            ],
            [
              18.4413496,
              59.395285466666664
            ],
            [
              18.441647433333333,
              59.39523346666667
            ],
            [
              18.442600966666667,
              59.3951386
            ],
            [
              18.444026066666666,
              59.39513756666667
            ],
            [
              18.4455927,
              59.39534476666667
            ],
            [
              18.447083533333334,
              59.395680766666665
            ],
            [
              18.448015166666668,
              59.396002
            ],
            [
              18.4484295,
              59.39623626666667
            ],
            [
              18.44860003333333,
              59.3963834
            ],
            [
              18.448640033333334,
              59.3964203
            ],
            [
              18.448642166666666,
              59.3964202
            ],
            [
              18.448647766666667,
              59.396423
            ],
            [
              18.448567133333334,
              59.39640213333333
            ],
            [
              18.4478306,
              59.39611576666667
            ],
            [
              18.4466424,
              59.395635166666665
            ],
            [
              18.4450621,
              59.39527486666667
            ],
            [
              18.443347366666668,
              59.3950673
            ],
            [
              18.4421094,
              59.395087933333336
            ],
            [
              18.4415366,
              59.39522216666667
            ],
            [
              18.441349966666667,
              59.395294766666666
            ],
            [
              18.4413408,
              59.3952986
            ],
            [
              18.4413404,
              59.39529973333333
            ],
            [
              18.4413452,
              59.39529386666667
            ],
            [
              18.4413395,
              59.3952873
            ],
            [
              18.441338933333334,
              59.395287
            ],
            [
              18.4413414,
              59.395287
            ],
            [
              18.441329933333332,
              59.3952876
            ],
            [
              18.441322333333332,
              59.395293333333335
            ],
            [
              18.441323466666667,
              59.39529773333334
            ],
            [
              18.441328133333332,
              59.3952944
            ],
            [
              18.441328666666667,
              59.39529726666667
            ],
            [
              18.441320533333332,
              59.3953
            ],
            [
              18.4413502666666

## `get_trips_from_vessel_data`

Processes vessel data to extract 'trips' between 'stops'. A 'stop' is defined as a period of time where the vessel's speed is below a certain threshold. The function removes all the measurements that correspond to speeds below the defined threshold and then splits the data into trips where the time gaps exceed the defined time threshold. The function defaults to a speed threshold of 1 knot and a time threshold of 1 minute. A trip is a dictionary containing keys-value corresponding to the trips path, timestamps, and correponding measurements. 

The example below demonstrates.

    NOTE: `get_trips_from_vessel_data` assumes that the timestamps in the provided vessel data are in UTC and transforms them to CET. See the function's documentation for the default argument `time_zone`.

In [13]:
# Fetch vessel data, averaged within a 30 seconds time bucket
vessel_data = fetch_vessel_data(
    vessel_id, 
    "2024-11-01 12:00:00",
    "2024-11-01 12:30:00",
    time_bucket="30 seconds",
    parameter_ids=["positioningsystem_latitude_deg_1", "positioningsystem_longitude_deg_1", "sog", "fuelcons"]
)

trips = get_trips_from_vessel_data(vessel_data)
print(f"There are {len(trips)} trips in the data.")

# Constructing the keys output
trip_keys_str = "\n\t".join(trips[0].keys())
print(f"The keys of the first trip are:\n\t{trip_keys_str}")

There are 4 trips in the data.
The keys of the first trip are:
	path
	enginemain_fuelcons_lph_1
	enginemain_fuelcons_lph_2
	enginemain_fuelcons_lph_3
	enginemain_fuelcons_lph_4
	positioningsystem_sog_kn_1
	time


In [14]:
# Make a list of paths
paths = [trip["path"] for trip in trips]

# Plot the paths
plot_paths(paths)

{
  "initialViewState": {
    "latitude": 59.395482132000005,
    "longitude": 18.44524827866667,
    "zoom": 15
  },
  "layers": [
    {
      "@@type": "PathLayer",
      "data": [
        {
          "color": [
            31,
            119,
            180
          ],
          "path": [
            [
              18.441647433333333,
              59.39523346666667
            ],
            [
              18.442600966666667,
              59.3951386
            ],
            [
              18.444026066666666,
              59.39513756666667
            ],
            [
              18.4455927,
              59.39534476666667
            ],
            [
              18.447083533333334,
              59.395680766666665
            ],
            [
              18.448015166666668,
              59.396002
            ],
            [
              18.4484295,
              59.39623626666667
            ]
          ]
        },
        {
          "color": [
            255,
            127,
            14
          ],
          "path": [
            [
              18.4478306,
              59.39611576666667
            ],
            [
              18.4466424,
              59.395635166666665
            ],
            [
              18.4450621,
              59.39527486666667
            ],
            [
              18.443347366666668,
              59.3950673
            ],
            [
              18.4421094,
              59.395087933333336
            ],
            [
              18.4415366,
              59.39522216666667
            ]
          ]
        },
        {
          "color": [
            44,
            160,
            44
          ],
          "path": [
            [
              18.441816,
              59.39516183333333
            ],
            [
              18.443006933333333,
              59.39489566666666
            ],
            [
              18.4446364,
              59.394819033333334
            ],
            [
              18.4462557,
              59.395058066666664
            ],
            [
              18.447551533333332,
              59.3955716
            ],
            [
              18.448288966666667,
              59.39606346666667
            ],
            [
              18.448583366666668,
              59.39635953333333
            ]
          ]
        },
        {
          "color": [
            214,
            39,
            40
          ],
          "path": [
            [
              18.448293366666668,
              59.39629373333333
            ],
            [
              18.447105,
              59.39585233333333
            ],
            [
              18.4454979,
              59.39545833333333
            ],
            [
              18.443811233333335,
              59.395220433333336
            ],
            [
              18.442436733333334,
              59.39512263333334
            ]
          ]
        }
      ],
      "getColor": "@@=color",
      "id": "b1e7468a-14a0-417d-927b-f636f2561490",
      "opacity": 0.95,
      "rounded": true,
      "widthMinPixels": 5
    }
  ],
  "mapProvider": "carto",
  "mapStyle": "https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json",
  "views": [
    {
      "@@type": "MapView",
      "controller": true
    }
  ]
}

## `cluster_paths`

Cluster paths based on their Fréchet distance and direction similarity. In other words, tries to cluster all the paths based on how similar they are an the overall direction on which the vessel is travelling. The example below demonstrates.

In [15]:
clusters = cluster_paths(paths)
colors = get_cluster_colors(clusters)
plot_paths(paths, colors=colors)


{
  "initialViewState": {
    "latitude": 59.395482132000005,
    "longitude": 18.44524827866667,
    "zoom": 15
  },
  "layers": [
    {
      "@@type": "PathLayer",
      "data": [
        {
          "color": [
            31,
            119,
            180
          ],
          "path": [
            [
              18.441647433333333,
              59.39523346666667
            ],
            [
              18.442600966666667,
              59.3951386
            ],
            [
              18.444026066666666,
              59.39513756666667
            ],
            [
              18.4455927,
              59.39534476666667
            ],
            [
              18.447083533333334,
              59.395680766666665
            ],
            [
              18.448015166666668,
              59.396002
            ],
            [
              18.4484295,
              59.39623626666667
            ]
          ]
        },
        {
          "color": [
            255,
            127,
            14
          ],
          "path": [
            [
              18.4478306,
              59.39611576666667
            ],
            [
              18.4466424,
              59.395635166666665
            ],
            [
              18.4450621,
              59.39527486666667
            ],
            [
              18.443347366666668,
              59.3950673
            ],
            [
              18.4421094,
              59.395087933333336
            ],
            [
              18.4415366,
              59.39522216666667
            ]
          ]
        },
        {
          "color": [
            31,
            119,
            180
          ],
          "path": [
            [
              18.441816,
              59.39516183333333
            ],
            [
              18.443006933333333,
              59.39489566666666
            ],
            [
              18.4446364,
              59.394819033333334
            ],
            [
              18.4462557,
              59.395058066666664
            ],
            [
              18.447551533333332,
              59.3955716
            ],
            [
              18.448288966666667,
              59.39606346666667
            ],
            [
              18.448583366666668,
              59.39635953333333
            ]
          ]
        },
        {
          "color": [
            255,
            127,
            14
          ],
          "path": [
            [
              18.448293366666668,
              59.39629373333333
            ],
            [
              18.447105,
              59.39585233333333
            ],
            [
              18.4454979,
              59.39545833333333
            ],
            [
              18.443811233333335,
              59.395220433333336
            ],
            [
              18.442436733333334,
              59.39512263333334
            ]
          ]
        }
      ],
      "getColor": "@@=color",
      "id": "13a7d20b-221b-465f-b56a-464ec7535f9c",
      "opacity": 0.95,
      "rounded": true,
      "widthMinPixels": 5
    }
  ],
  "mapProvider": "carto",
  "mapStyle": "https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json",
  "views": [
    {
      "@@type": "MapView",
      "controller": true
    }
  ]
}

## Other

Addtional functions are available in `utils.py`, take a look at the file for more information.